In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
#os.remove('/kaggle/working/housing.csv.2')
data='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
!wget $data

df_org = pd.read_csv('housing.csv')

--2022-09-25 12:40:57--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.07s   

2022-09-25 12:40:57 (19.8 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [3]:
columns = ['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value',
'ocean_proximity']
columns

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [4]:
df = df_org[columns].fillna(0)
df['rooms_per_household'] = df.total_rooms / df.households
df['bedrooms_per_room'] = df.total_bedrooms / df.total_rooms
df['population_per_household'] = df.population / df.households

df.head().T
# df_org.head().T

,0,1,2,3,4
latitude,37.88,37.86,37.85,37.85,37.85
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


## Question 1
What is the most frequent observation (mode) for the column ocean_proximity?

In [5]:
df.ocean_proximity.mode()

0    <1H OCEAN
dtype: object

## Make median_house_value binary
* We need to turn the median_house_value variable from numeric into binary.
* Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.

In [6]:
# binarizing median_house_value under new name 'above_average'
df['above_average'] = (df.median_house_value > df.median_house_value.mean()).astype(int)

## Split the data
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value (median_house_value) is not in your dataframe.

In [7]:
# Splitting the data set

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

print(len(df_train), len(df_val), len(df_test))

df_train.reset_index()
df_val.reset_index()
df_test.reset_index()

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

for c in ['median_house_value']:
    del df_train[c]
    del df_val[c]
    del df_test[c]

12384 4128 4128


## Question 2
* Create the correlation matrix for the numerical features of your **train** dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

In [8]:
df.dtypes

latitude                    float64
longitude                   float64
housing_median_age          float64
total_rooms                 float64
total_bedrooms              float64
population                  float64
households                  float64
median_income               float64
median_house_value          float64
ocean_proximity              object
rooms_per_household         float64
bedrooms_per_room           float64
population_per_household    float64
above_average                 int64
dtype: object

In [9]:
numerical = list((df.dtypes[df.dtypes != 'object'].index))

# remove target variables
numerical.remove('median_house_value') 
numerical.remove('above_average')

numerical

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [10]:
categorical = list((df.dtypes[df.dtypes == 'object'].index))
categorical

['ocean_proximity']

In [11]:
corr_matrix = list()

for c in numerical:
    for o in numerical:
        if c == o: continue
        corr_matrix.append({'name' : '%s___%s' % (c, o),
                      'corr' : df_train[c].corr(df_train[o])})

sorted(corr_matrix, key = lambda d: d['corr'], reverse=True)[0]

{'name': 'total_bedrooms___households', 'corr': 0.979399352769416}

## Question 3
* Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
* What is the value of mutual information?
* Round it to 2 decimal digits using round(score, 2)

In [12]:
mutual_info_score(df_train.above_average, df_train.ocean_proximity).round(2)

0.1

## Question 4
* Now let's train a logistic regression
* Remember that we have one categorical variable `ocean_proximity` in the data. Include it using one-hot encoding.
* Fit the model on the training dataset.
  * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  * `model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [13]:
dicts_train = df_train[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)

In [14]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

model.fit(X_train, y_train) #<-- training the model

w = list(model.coef_[0])
w0 = model.intercept_[0]
(w0, w)

(-0.083829188440655,
 [0.18093530759982326,
  0.003954758787947762,
  0.03573955469112594,
  0.11598325702417112,
  0.08716280496403875,
  1.2120866441523424,
  0.473101980247531,
  -1.7147713304052514,
  0.01882696615444871,
  0.29153725950298887,
  0.8474759360596523,
  -0.0016371871780485171,
  0.010323040002691304,
  -0.014052207665170789,
  0.0018753074674865912,
  -0.00014597082449926822])

In [15]:
# Calculate the predictions for the validation data set using the model trained with training data

dicts_val = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(dicts_val)
val_pred = model.predict_proba(X_val)[:,1] >= 0.5

# Calculate relative frequency
val_pred_accuracy = (val_pred == y_val).mean()
val_pred_accuracy.round(2)

0.84

## Question 5

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
* Which of following feature has the smallest difference?
  * `total_rooms`
  * `total_bedrooms`
  * `population`
  * `households`
**note**: the difference doesn't have to be positive

In [16]:
columns = categorical + numerical
tlist = list()

for c in columns:
    tolumns = columns.copy()
    tolumns.remove(c)
    
    # hot-one encoding train data
    dicts_train_small = df_train[tolumns].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train_small = dv.fit_transform(dicts_train_small)
    
    # traning the model with train data
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_small, y_train) # training the model with the subset of features
    
    # hot-one encoding validation data
    dicts_val_small = df_val[tolumns].to_dict(orient='records')
    X_val_small = dv.transform(dicts_val_small)
    
    # calculate prediction using the trained model
    val_small_pred = model.predict_proba(X_val_small)[:,1] >= 0.5
    val_small_pred
    
    # Calculate relative frequency
    val_small_pred_accuracy = (val_small_pred == y_val).mean()
    
    # find diff with original accuracy from full validation dataset
    diff = val_small_pred_accuracy - val_pred_accuracy
    tlist.append({'col' : c,  'accuracy' : abs(diff)}) # abs to remove sign

sorted(tlist, key = lambda d: d['accuracy'])

# total_rooms is the answer, as population_per_household is not in optional list

[{'col': 'total_bedrooms', 'accuracy': 0.0004844961240310086},
 {'col': 'population_per_household', 'accuracy': 0.0004844961240310086},
 {'col': 'total_rooms', 'accuracy': 0.0007267441860464574},
 {'col': 'rooms_per_household', 'accuracy': 0.0007267441860465684},
 {'col': 'bedrooms_per_room', 'accuracy': 0.0014534883720930258},
 {'col': 'latitude', 'accuracy': 0.002422480620155043},
 {'col': 'households', 'accuracy': 0.0031492248062016115},
 {'col': 'housing_median_age', 'accuracy': 0.004844961240310086},
 {'col': 'longitude', 'accuracy': 0.005087209302325646},
 {'col': 'population', 'accuracy': 0.009932170542635732},
 {'col': 'ocean_proximity', 'accuracy': 0.016715116279069853},
 {'col': 'median_income', 'accuracy': 0.04990310077519389}]

## Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'median_house_value'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (`model = Ridge(alpha=a, solver="sag", random_state=42)`) on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [17]:
# Splitting the data set

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

print(len(df_train), len(df_val), len(df_test))

df_train.reset_index()
df_val.reset_index()
df_test.reset_index()

y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

for c in ['median_house_value']:
    del df_train[c]
    del df_val[c]
    del df_test[c]

12384 4128 4128


In [18]:
def get_rmse(y_pred, y):
    se = np.square( y_pred - y)
    mse = se.mean()
    return np.sqrt(mse)

In [19]:
from sklearn.linear_model import Ridge

dicts_train = df_train[categorical + numerical].to_dict(orient='records')
tlist =list()

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)

for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    # model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train) #<-- training the model
    
    dicts_val = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(dicts_val)
    y_pred = model.predict(X_val)
    rmse = get_rmse(y_pred, y_val)
    tlist.append({'alpha' : a,  'rmse' : round(rmse, 3)}) #rmse rounded to 3 decimal places
    
sorted(tlist, key = lambda d: d['rmse'])


[{'alpha': 0, 'rmse': 0.524},
 {'alpha': 0.01, 'rmse': 0.524},
 {'alpha': 0.1, 'rmse': 0.524},
 {'alpha': 1, 'rmse': 0.524},
 {'alpha': 10, 'rmse': 0.524}]